In [43]:
######################################################
# Additional Lib
######################################################
import datetime
import os
from typing import Callable, Optional
import pandas as pd
import json
import warnings
warnings.filterwarnings("ignore")
######################################################
# Data Visualizations
######################################################
import matplotlib.pyplot as plt #type: ignore
import matplotlib as matplot #type: ignore
import seaborn as sns #type: ignore

In [44]:
pd.set_option('display.max_columns', None)
path = './data/dataset.csv'
df = pd.read_csv(path)
df.head(10)
# df = df.head(500000)

print("\n\n\n")
print("DATASET LENGTH")
print("ROWS: ", df.shape[0])
print("COLUMNS: ", df.shape[1])





DATASET LENGTH
ROWS:  5078345
COLUMNS:  11


In [45]:
df.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0


In [46]:
###################################################################################################
# Data Visualization Function - Helpers
###################################################################################################

def format_title_output(text):
    return f"{'++' * 50}\n{text.upper()}\n{'++' * 50}\n"


#Null data with heatmap
def create_null_data_heatmap(
    title=None,
    rotate='horizontal',
    dataset=None
):
    try:
        plt.figure(figsize=(15,5))
        sns.heatmap(dataset.isnull(), cbar=False)
        plt.title(title, fontweight='bold')
        plt.xticks(rotation=rotate)
        plt.show()
    except Exception as error:
        print("**"*50)
        print("ERROR TO CREATE THE HEATMAP: ", error)
        print("**"*50)


#Null data with heatmap
def create_correlation_heatmap(
    title=None,
    dataset=None
):
    try:
        plt.figure(figsize=(15,8))
        correlation = dataset.corr()
        correlation = (correlation)
        sns.heatmap(correlation,
                    xticklabels=correlation.columns.values,
                    yticklabels=correlation.columns.values,
                    annot=True
        )
        plt.title(title, fontweight='bold')
        plt.show()
    except Exception as error:
        print("**"*50)
        print("ERROR TO CREATE THE HEATMAP: ", error)
        print("**"*50)

def create_boxplots(data, columns, ncols=2, figsize=(25, 6)):
    nrows = (len(columns) + ncols - 1) // ncols
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    axes = axes.flatten() if nrows > 1 else axes
    for i, col in enumerate(columns):
        sns.boxplot(y=data[col], ax=axes[i])
        axes[i].set_title(col)
    for j in range(len(columns), len(axes)):
        axes[j].axis('off')
    plt.tight_layout()
    plt.show()


def create_histogram(data, column, figsize=(15, 4), color="g", kde=False):
    try:
        plt.figure(figsize=figsize)
        sns.histplot(data[column], kde=kde, color=color).set_title(f'{column[0]} Distribution', fontweight='bold')
        plt.title(f"{column[0]} Distribution", fontweight='bold')
        plt.show()
    except Exception as error:
        print("**"*50)
        print("ERROR TO CREATE THE HISTOGRAM: ", error)
        print("**"*50)

###################################################################################################
# Data Preprocessing - Helpers
###################################################################################################
def detect_outliers(data, columns):
    outlier_info = {}
    for col in columns:
        Q1 = data[col].quantile(0.25)  # First quartile (25th percentile)
        Q3 = data[col].quantile(0.75)  # Third quartile (75th percentile)
        IQR = Q3 - Q1  # Interquartile range
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
        max_outlier = outliers[col].max() if not outliers.empty else None

        outlier_info[col] = {
            "max_outlier_value": max_outlier,
            "number_of_outliers": outliers.shape[0],
            "lower_bound": lower_bound,
            "upper_bound": upper_bound,
        }

    return outlier_info

In [47]:
# Check the dataset info
print(format_title_output("DATASET INFO"))
print(df.info())

print("\n\n\n")
print(format_title_output("DATASET LENGTH"))
print("ROWS: ", df.shape[0])
print("COLUMNS: ", df.shape[1])

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
DATASET INFO
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078345 entries, 0 to 5078344
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Timestamp           object 
 1   From Bank           int64  
 2   Account             object 
 3   To Bank             int64  
 4   Account.1           object 
 5   Amount Received     float64
 6   Receiving Currency  object 
 7   Amount Paid         float64
 8   Payment Currency    object 
 9   Payment Format      object 
 10  Is Laundering       int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 426.2+ MB
None




++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
DATASET LENGTH
+++++++++++++++++++++++++++++++++++++++++++++++++++++

In [48]:
# rename the columns with lowercase and _ underscore if there is a space
print(format_title_output("DATASET NEW COLUMNS"))

df.columns = [col.lower().replace(" ", "_") for col in df.columns]
for i,j in enumerate(df.columns):
    print(i, " : ", j)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
DATASET NEW COLUMNS
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

0  :  timestamp
1  :  from_bank
2  :  account
3  :  to_bank
4  :  account.1
5  :  amount_received
6  :  receiving_currency
7  :  amount_paid
8  :  payment_currency
9  :  payment_format
10  :  is_laundering


In [49]:
# Convert the timestamp column to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%dT%H:%M:%S')


# Get the min and max datetime
min_time = df['timestamp'].min()
max_time = df['timestamp'].max()

print("Min Datetime:", min_time)
print("Max Datetime:", max_time)

Min Datetime: 2022-09-01T00:00:00
Max Datetime: 2022-09-18T16:18:00


In [50]:
amount_consistency = df.loc[~(df['amount_received'] == df['amount_paid'])]
ccy_consistency = df.loc[~(df['receiving_currency'] == df['payment_currency'])]

print(format_title_output("AMOUNT CONSISTENCY - CREDIT & DEBIT"))
print(amount_consistency.shape)

print(format_title_output("CURRENCY CONSISTENCY - CREDIT & DEBIT"))
print(ccy_consistency.shape)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
AMOUNT CONSISTENCY - CREDIT & DEBIT
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

(72158, 11)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
CURRENCY CONSISTENCY - CREDIT & DEBIT
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

(72170, 11)


In [51]:
#Check the currency

print(format_title_output("RECEIVED CURRENCY"))
print(df['receiving_currency'].unique())
print("\n\n")

print(format_title_output("PAYMENT CURRENCY"))
print(df['payment_currency'].unique())

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
RECEIVED CURRENCY
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

['US Dollar' 'Bitcoin' 'Euro' 'Australian Dollar' 'Yuan' 'Rupee'
 'Mexican Peso' 'Yen' 'UK Pound' 'Ruble' 'Canadian Dollar' 'Swiss Franc'
 'Brazil Real' 'Saudi Riyal' 'Shekel']



++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
PAYMENT CURRENCY
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

['US Dollar' 'Bitcoin' 'Euro' 'Australian Dollar' 'Yuan' 'Rupee' 'Yen'
 'Mexican Peso' 'UK Pound' 'Ruble' 'Canadian Dollar' 'Swiss Franc'
 'Brazil Real' 'Saudi Riyal' 'Shekel']


In [53]:
negative_df = df[df['amount_paid'] < 0]
negative_df

,timestamp,from_bank,account,to_bank,account.1,amount_received,receiving_currency,amount_paid,payment_currency,payment_format,is_laundering


In [54]:
df.head()
print(df.describe(include='all'))

                  timestamp     from_bank    account       to_bank  account.1  \
count               5078345  5.078345e+06    5078345  5.078345e+06    5078345   
unique                15018           NaN     496995           NaN     420636   
top     2022-09-01T00:04:00           NaN  100428660           NaN  100428660   
freq                  11193           NaN     168672           NaN       1084   
mean                    NaN  4.573057e+04        NaN  6.574456e+04        NaN   
std                     NaN  8.176562e+04        NaN  8.409299e+04        NaN   
min                     NaN  1.000000e+00        NaN  1.000000e+00        NaN   
25%                     NaN  1.190000e+02        NaN  4.259000e+03        NaN   
50%                     NaN  9.679000e+03        NaN  2.156800e+04        NaN   
75%                     NaN  2.862800e+04        NaN  1.223320e+05        NaN   
max                     NaN  3.563030e+05        NaN  3.562940e+05        NaN   

        amount_received rec

In [55]:
# Check the outlier for the amount of each transaction
# Function to convert to USD
# from forex_python.converter import CurrencyRates
exchange_rates = {
    'Australian Dollar': 0.62124,  # AUD
    'Bitcoin': 23000.0,            # BTC (example rate)
    'Brazil Real': 0.1980,         # BRL
    'Canadian Dollar': 0.6876,     # CAD
    'Euro': 1.0362,                # EUR
    'Mexican Peso': 0.04835,       # MXN
    'Ruble': 0.01636,              # RUB
    'Rupee': 0.01153,              # INR
    'Saudi Riyal': 0.2667,         # SAR
    'Shekel': 0.2840,              # ILS
    'Swiss Franc': 1.0989,         # CHF
    'UK Pound': 1.2401,            # GBP
    'US Dollar': 1.0,              # USD
    'Yen': 0.006443,               # JPY
    'Yuan': 0.1391                 # CNY
}

# Function to convert to USD
def convert_to_usd(currency, amount):
    if currency in exchange_rates:
        return amount * exchange_rates[currency]
    else:
        print(f"Exchange rate for {currency} not found.")
        return amount

def create_bar_plot(df, column_name, title=None):
    plt.figure(figsize=(20, 5))
    sns.countplot(x=column_name, data=df, order=df[column_name].value_counts().index, palette='Set2')
    plt.title(title, fontweight='bold', color='#065164')
    plt.show()
    print(format_title_output(title.upper() + " VALUE COUNTS"))
    print(df[column_name].value_counts())

def create_pie_plot(df, column_name, title=None):
    plt.figure(figsize=(8, 8))
    df[column_name].value_counts().plot.pie(autopct='%1.1f%%', colors=sns.color_palette('Set2', len(df[column_name].value_counts())))
    plt.title(title, fontweight='bold', color='#065164')
    plt.show()
    print(format_title_output(title.upper() + " VALUE COUNTS"))
    print(df[column_name].value_counts())

def create_kde_plot(df, column_name, title=None, color='g'):
    plt.figure(figsize=(15, 5))
    sns.kdeplot(df[column_name], shade=True, color=color)
    plt.title(title, fontweight='bold', color='#065164')
    plt.show()

def create_kde_plot_with_log(df, column_name, title=None, color='r'):
    log_column = np.log1p(df[column_name])  # log(x+1) to avoid issues with zero values
    plt.figure(figsize=(15, 5))
    sns.kdeplot(log_column, shade=True, color=color)
    plt.title(title, fontweight='bold', color='#065164')
    plt.show()
    print(format_title_output(title.upper()))
    print(log_column.describe())

def create_histogram_with_bins(df, column_name, title=None, color='r', bins=20):
    plt.figure(figsize=(15, 5))
    sns.histplot(df[column_name], bins=bins, kde=True, color=color)
    plt.title(title, fontweight='bold', color='#065164')
    plt.show()

    print(format_title_output(title.upper()))
    print(df[column_name].describe())

def create_transaction_line_plot(df, date_column, amount_column, title=None):
    df[date_column] = pd.to_datetime(df[date_column])
    daily_transactions = df.groupby(df[date_column].dt.date)[amount_column].sum().reset_index()
    plt.figure(figsize=(15, 5))
    sns.lineplot(x=daily_transactions[date_column], y=daily_transactions[amount_column],
                 marker='o', color='b', linewidth=2, markersize=8, label='Transaction Amount')
    plt.title(title, fontweight='bold', color='#065164', fontsize=12)
    plt.xlabel('Date', fontsize=12, color='#065164')
    plt.ylabel('Total Transaction Amount', fontweight='bold', fontsize=12, color='#065164')
    plt.show()


In [56]:
# Convert to the USD currency based on the exchange rate
df['usd_amount'] = df.apply(lambda row: convert_to_usd(row['payment_currency'], row['amount_paid']), axis=1)
none_us_dollar = df.loc[(df['payment_currency'] != 'US Dollar')]
none_us_dollar.head()

,timestamp,from_bank,account,to_bank,account.1,amount_received,receiving_currency,amount_paid,payment_currency,payment_format,is_laundering,usd_amount
1155,2022-09-01T00:16:00,220,8001C8C51,1420,8003093C1,0.025852,Bitcoin,0.025852,Bitcoin,Bitcoin,0,594.596000
1174,2022-09-01T00:22:00,1362,80030A870,11,80064C9B0,52.110000,Euro,52.110000,Euro,Credit Card,0,53.996382
1467,2022-09-01T00:15:00,1,80005C0A1,1588,8003AC471,0.016891,Bitcoin,0.016891,Bitcoin,Bitcoin,0,388.493000
3940,2022-09-01T00:08:00,513,8006538E1,1688,8006DD361,1.621978,Bitcoin,1.621978,Bitcoin,Bitcoin,0,37305.494000
6302,2022-09-01T00:19:00,1688,800646381,1674,800B1B1B1,0.045449,Bitcoin,0.045449,Bitcoin,Bitcoin,0,1045.327000


In [57]:
is_laundering_class_1 = df[df['is_laundering'] == 1].head(500)
is_laundering_class_0 = df[df['is_laundering'] == 0].head(500)
print(is_laundering_class_1.head())
print(is_laundering_class_0.head())

                 timestamp  from_bank    account  to_bank  account.1  \
4742   2022-09-01T00:21:00         70  100428660     1124  800825340   
40621  2022-09-01T00:03:00         70  100428660    11474  805B716C0   
69152  2022-09-01T00:01:00         70  100428660    15980  80B39E7B0   
85763  2022-09-01T00:03:00         70  100428660   113798  80DC756E0   
90684  2022-09-01T00:23:00         70  100428660    32375  80E480620   

       amount_received receiving_currency  amount_paid payment_currency  \
4742         389769.39          US Dollar    389769.39        US Dollar   
40621         29024.33          US Dollar     29024.33        US Dollar   
69152           792.92          US Dollar       792.92        US Dollar   
85763      13171425.53          US Dollar  13171425.53        US Dollar   
90684         14288.83          US Dollar     14288.83        US Dollar   

      payment_format  is_laundering   usd_amount  
4742          Cheque              1    389769.39  
40621    Credi

In [58]:
# Check the dataset info
print(format_title_output("CLASS 1"))
print(len(is_laundering_class_1.account.value_counts()))
print(format_title_output("CLASS 0"))
print(len(is_laundering_class_0.account.value_counts()))

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
CLASS 1
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

336
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
CLASS 0
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

381


In [59]:
mimic_df = pd.concat([is_laundering_class_1, is_laundering_class_0], axis=0)
mimic_df.head()

,timestamp,from_bank,account,to_bank,account.1,amount_received,receiving_currency,amount_paid,payment_currency,payment_format,is_laundering,usd_amount
4742,2022-09-01T00:21:00,70,100428660,1124,800825340,389769.39,US Dollar,389769.39,US Dollar,Cheque,1,389769.39
40621,2022-09-01T00:03:00,70,100428660,11474,805B716C0,29024.33,US Dollar,29024.33,US Dollar,Credit Card,1,29024.33
69152,2022-09-01T00:01:00,70,100428660,15980,80B39E7B0,792.92,US Dollar,792.92,US Dollar,Credit Card,1,792.92
85763,2022-09-01T00:03:00,70,100428660,113798,80DC756E0,13171425.53,US Dollar,13171425.53,US Dollar,Cheque,1,13171425.53
90684,2022-09-01T00:23:00,70,100428660,32375,80E480620,14288.83,US Dollar,14288.83,US Dollar,Cash,1,14288.83


In [60]:
# Check the dataset info
print(format_title_output("DATASET INFO"))
print(df.info())

print("\n\n\n")
print(format_title_output("DATASET LENGTH"))
print("ROWS: ", mimic_df.shape[0])
print("COLUMNS: ", mimic_df.shape[1])

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
DATASET INFO
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078345 entries, 0 to 5078344
Data columns (total 12 columns):
 #   Column              Dtype  
---  ------              -----  
 0   timestamp           object 
 1   from_bank           int64  
 2   account             object 
 3   to_bank             int64  
 4   account.1           object 
 5   amount_received     float64
 6   receiving_currency  object 
 7   amount_paid         float64
 8   payment_currency    object 
 9   payment_format      object 
 10  is_laundering       int64  
 11  usd_amount          float64
dtypes: float64(3), int64(3), object(6)
memory usage: 464.9+ MB
None




++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
DATASET LENGTH
++++++++++++++++++++

In [62]:
mimic_df.to_csv('./data/minic_data.csv', index=False)